In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# probability model
from scipy.stats import multivariate_normal
from sklearn.mixture import GaussianMixture
import warnings
#warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 

In [ ]:
train_data = np.memmap("../../../C2FPL/concat_UCF.npy", shape=(779951,10,2048)) #UCF
# train_data = np.load("/home/anas.al-lahham/AD_Unsupervised/concat_XD_I3D.npy") #XD
train_data.shape

nalist = np.load("../../../C2FPL/list/nalist.npy") #UCF
# nalist = np.load("nalist_XD.npy") #XD

(779951, 10, 2048)

In [ ]:
new_repr = []
for i, (fromid, toid) in enumerate(nalist):
    new_repr.append(train_data[fromid:toid])

len(new_repr)
new_repr[i].shape

1609

In [ ]:
def estimate_gauss(X):
    m = X.shape[0]   # using only first dimension as we know it has only one feature - l2 norm
    mu = np.mean(X, axis=0)
    var = np.cov(X.T)
    return mu, var

def covariance_mat(X):
    X = np.mean(X , axis= 1)
    X =  X.transpose(1,0)
    cov  = np.cov(X)
    return cov

def get_matrix(data):
    l2_norm = np.sum(np.square(data), axis=2)
    n_train_crop_l2_norm_mean = np.mean(l2_norm, axis= 1)
    return n_train_crop_l2_norm_mean


def diff_l2(new_repr):
    l2_norms = []
    for i in range(len(new_repr)):
        l2_norms.append(get_matrix(new_repr[i]))
    mean_v_l2 = []
    for i in range(len(l2_norms)):
        mean_v_l2.append(np.diff(l2_norms[i], n=1).max())
    return mean_v_l2

In [ ]:
params = []
for i in range(len(new_repr)):
    param = get_matrix(new_repr[i])
    mu, var = estimate_gauss(param)
    params.append((mu, var, ))

np.array(params).shape

In [ ]:
gmm = GaussianMixture(n_components=2, max_iter=150, random_state=0, covariance_type='spherical')
# gmm_scores = gmm.score_samples(params)
labels = gmm.fit_predict(params)

y_gmm = gmm.fit_predict(params)
print(y_gmm.sum(), y_gmm.sum() / len(y_gmm))

score = y_gmm 
score = gmm.score_samples(params) 
pct_threshold = np.percentile(score, 3)
print(f'The threshold of the score is {pct_threshold:.2f}') 

res = np.array([1 if x < pct_threshold else 0 for x in score]) 
print(res.sum())

278 0.17277812305779988
The threshold of the score is -42.92
49


In [12]:
abnormal_portion = np.where(labels == 1)[0]
normal_portion = np.where(labels == 0)[0]
normal_portion.shape, abnormal_portion.shape

((1331,), (278,))

In [13]:
n_params = np.array(params)[normal_portion]
a_params = np.array(params)[abnormal_portion]
n_params.shape, a_params.shape

((1331, 2), (278, 2))

In [ ]:
abag = list(zip(list(np.array(params)[abnormal_portion]), abnormal_portion))
nbag = list(zip(list(np.array(params)[normal_portion]), normal_portion))
len(abag), len(nbag)

(278, 1331)

In [ ]:
nu = 1.0
step = 1

import time
start = time.time()

while len(abag) / len(nbag) < nu:
    temp_bag = nbag
    
    y_gmm = gmm.fit_predict([list(x[0]) for x in np.array(temp_bag)])
    score = y_gmm 
    score = gmm.score_samples([list(x[0]) for x in np.array(temp_bag)])
    pct_threshold = np.percentile(score, 3) 
    res = np.array([1 if x < pct_threshold else 0 for x in score]) 
    print(f'The threshold of the score in step {step} is {pct_threshold:.2f}, abnormal part: {res.sum()}') 
    
    abnormal_portion = np.where(res == 1)[0]
    normal_portion = np.where(res == 0)[0]
    
    abag += [(x[0], x[1]) for x in np.array(temp_bag)[abnormal_portion]]
    nbag = [(x[0], x[1]) for x in np.array(temp_bag)[normal_portion]]

    step += 1

print(time.time() - start)

The threshold of the score in step 1 is -37.58, abnormal part: 40


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (1331, 2) + inhomogeneous part.

In [17]:
print(np.where(np.array([x[1] for x in abag]) < 810)[0].shape, len([x[1] for x in abag]))
print('correctness acc: ', np.where(np.array([x[1] for x in abag]) < 810)[0].shape[0] / len([x[1] for x in abag]))

(171,) 278
correctness acc:  0.6151079136690647


In [18]:
print(np.where(np.array([x[1] for x in nbag]) > 810)[0].shape, len([x[1] for x in nbag]))
print('correctness acc: ', np.where(np.array([x[1] for x in nbag]) > 810)[0].shape[0] / len([x[1] for x in nbag]))

(691,) 1331
correctness acc:  0.5191585274229902


In [19]:
temp = [k[1] for k in sorted([(x[1], 1.0) for x in abag] + [(x[1], 0.0) for x in nbag], key=lambda z: z[0])]
sum(temp), len(temp)

(278.0, 1609)

In [ ]:
# normal set creation
normal_set = {}
for i in range(len(new_repr)):
    if temp[i] == 0.0:
        normal_set[i] = new_repr[i]

# abnormal set creation
abnormal_set = {}
for i in range(len(new_repr)):
    if temp[i] == 1.0:
        abnormal_set[i] = new_repr[i]
        
len(normal_set), len(abnormal_set)

In [ ]:
l2_norms_N = np.empty(0,)
for (idel, sample) in normal_set.items():
    # print(sample.shape)
    l2_norms_N = np.append(l2_norms_N,get_matrix(sample))

In [ ]:
mu_GMM, var_GMM = estimate_gauss(np.array(l2_norms_N))
# probability model
p = multivariate_normal(mu_GMM, var_GMM)

In [ ]:
ground_truth = {} 
length = 0.2 
for (idel, sample) in abnormal_set.items(): 
    # feature extraction 
    # sample_matrix = np.sum(np.square(sample), axis=1)  # for just l2 !!!!!
    sample_matrix = get_matrix(sample)
    
    # get p values
    probs = p.pdf(sample_matrix)
    temp_list = []
    temp_list += [0.0] * len(probs)
    
    window_size = int(len(probs) * length)  # fixed
    temp = []
    for idx in range(0, len(probs) - window_size + 1):
        arr = 0
        for i in range(idx, idx + window_size - 1):
            arr += abs(probs[i+1] - probs[i])
        temp.append(arr)

    for i in range(temp.index(max(temp)), temp.index(max(temp)) + window_size):
        temp_list[i] = 1.0

    ground_truth[idel] = temp_list

In [ ]:
final_gt = []
abnormal_gt = []
for i in range(len(new_repr)):
    if i in normal_set.keys():
        final_gt += [0.0] * new_repr[i].shape[0]
    else:
        final_gt += ground_truth[i]
        abnormal_gt+= ground_truth[i]
        
len(final_gt)

In [ ]:
np.save('unsupervised_PL/'+'UCF_labels_entropy_1.npy', final_gt) #UCF
# np.save('Unsup_labels/'+'XD_I3D_unsup_labels_10_V2_GMM.npy', final_gt) #XD